In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", future=True, echo=True)

from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2022-12-16 14:06:18,892 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:06:18,898 INFO sqlalchemy.engine.Engine select 'hello world'
2022-12-16 14:06:18,900 INFO sqlalchemy.engine.Engine [generated in 0.00904s] ()
[('hello world',)]
2022-12-16 14:06:18,904 INFO sqlalchemy.engine.Engine ROLLBACK


In [2]:
# "commit as you go"
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()


2022-12-16 14:06:18,993 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:06:18,996 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-12-16 14:06:18,999 INFO sqlalchemy.engine.Engine [generated in 0.00607s] ()
2022-12-16 14:06:19,003 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-12-16 14:06:19,005 INFO sqlalchemy.engine.Engine [generated in 0.00198s] [(1, 1), (2, 4)]
2022-12-16 14:06:19,008 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
# "begin once"
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )
# BEGIN (implicit)
# INSERT INTO some_table (x, y) VALUES (?, ?)
# [...] [(6, 8), (9, 10)]
# <sqlalchemy.engine.cursor.CursorResult object at 0x...>
# COMMIT

2022-12-16 14:06:19,166 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:06:19,168 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-12-16 14:06:19,171 INFO sqlalchemy.engine.Engine [cached since 0.1683s ago] [(6, 8), (9, 10)]
2022-12-16 14:06:19,174 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")
# BEGIN (implicit)
# SELECT x, y FROM some_table
# [...] ()
# x: 1  y: 1
# x: 2  y: 4
# x: 6  y: 8
# x: 9  y: 10
# ROLLBACK

2022-12-16 14:06:19,311 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:06:19,313 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2022-12-16 14:06:19,315 INFO sqlalchemy.engine.Engine [generated in 0.00446s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-12-16 14:06:19,319 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")
# BEGIN (implicit)
# SELECT x, y FROM some_table WHERE y > ?
# [...] (2,)
# x: 2  y: 4
# x: 6  y: 8
# x: 9  y: 10
# ROLLBACK

2022-12-16 14:06:19,429 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:06:19,431 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2022-12-16 14:06:19,433 INFO sqlalchemy.engine.Engine [generated in 0.00483s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-12-16 14:06:19,436 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")
# BEGIN (implicit)
# SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
# [...] (6,)
# x: 6  y: 8
# x: 9  y: 10
# x: 11  y: 12
# x: 13  y: 14
# ROLLBACK

2022-12-16 14:06:19,931 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:06:19,935 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-12-16 14:06:19,937 INFO sqlalchemy.engine.Engine [generated in 0.00230s] (6,)
x: 6  y: 8
x: 9  y: 10
2022-12-16 14:06:19,941 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()
# BEGIN (implicit)
# UPDATE some_table SET y=? WHERE x=?
# [...] [(11, 9), (15, 13)]
# COMMIT

2022-12-16 14:06:20,022 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:06:20,025 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2022-12-16 14:06:20,026 INFO sqlalchemy.engine.Engine [generated in 0.00179s] [(11, 9), (15, 13)]
2022-12-16 14:06:20,031 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
from sqlalchemy import MetaData
metadata_obj = MetaData()
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [9]:
user_table.c.name
# Column('name', String(length=30), table=<user_account>)

user_table.c.keys()
# ['id', 'name', 'fullname']

user_table.primary_key
# PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [10]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)


The MetaData object also features a MetaData.drop_all() method that will emit DROP statements in the reverse order as it would emit CREATE in order to drop schema elements.

In [11]:

metadata_obj.create_all(engine)
# BEGIN (implicit)
# PRAGMA main.table_...info("user_account")
# ...
# PRAGMA main.table_...info("address")
# ...
# CREATE TABLE user_account (
#     id INTEGER NOT NULL,
#     name VARCHAR(30),
#     fullname VARCHAR,
#     PRIMARY KEY (id)
# )
# ...
# CREATE TABLE address (
#     id INTEGER NOT NULL,
#     user_id INTEGER NOT NULL,
#     email_address VARCHAR NOT NULL,
#     PRIMARY KEY (id),
#     FOREIGN KEY(user_id) REFERENCES user_account (id)
# )
# ...
# COMMIT

2022-12-16 14:06:20,632 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:06:20,635 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-12-16 14:06:20,637 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:06:20,641 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-12-16 14:06:20,643 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:06:20,645 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-12-16 14:06:20,647 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:06:20,650 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-12-16 14:06:20,651 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:06:20,655 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-12-16 14:06:20,657 INFO sqlalchemy.engine.Engine [no key 0.00185s] ()
2022-12-16 14:06:20,661 INFO sqlalchemy.engine.Engine 
C

In [12]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [13]:
Base.metadata
# MetaData()
Base.registry
# <sqlalchemy.orm.decl_api.registry object at 0x...>

In [14]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]

    addresses: Mapped[List["Address"]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))

    user: Mapped[User] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [15]:
import sqlalchemy
sqlalchemy.__version__  

'2.0.0b4'

In [16]:
sandy = User(name="sandy", fullname="Sandy Cheeks")

In [17]:
Base.metadata.create_all(engine)

2022-12-16 14:38:36,158 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:38:36,159 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-12-16 14:38:36,160 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:38:36,164 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-12-16 14:38:36,165 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:38:36,168 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)
# BEGIN (implicit)
# PRAGMA main.table_...info("some_table")
# [raw sql] ()
# SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
# [raw sql] ('some_table',)
# PRAGMA main.foreign_key_list("some_table")
# ...
# PRAGMA main.index_list("some_table")
# ...
# ROLLBACK

2022-12-16 14:41:35,943 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:41:35,945 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-12-16 14:41:35,947 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:41:35,953 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2022-12-16 14:41:35,955 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-12-16 14:41:35,958 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-12-16 14:41:35,959 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:41:35,962 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-12-16 14:41:35,963 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-16 14:41:35,966 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [19]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

In [20]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")
session = Session(engine)


The objects are then added to the Session using the Session.add() method. When this is called, the objects are in a state known as pending and have not been inserted yet:

In [21]:
session.add(squidward)
session.add(krabs)

When we have pending objects, we can see this state by looking at a collection on the Session called Session.new:

In [22]:
session.new
# IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])

The Session makes use of a pattern known as unit of work. This generally means it accumulates changes one at a time, but does not actually communicate them to the database until needed. This allows it to make better decisions about how SQL DML should be emitted in the transaction based on a given set of pending changes. When it does emit SQL to the database to push out the current set of changes, the process is known as a flush.

We can illustrate the flush process manually by calling the Session.flush() method:

In [23]:
session.flush()
# BEGIN (implicit)
# INSERT INTO user_account (name, fullname) VALUES (?, ?), (?, ?) RETURNING id
# [...] ('squidward', 'Squidward Tentacles', 'ehkrabs', 'Eugene H. Krabs')

2022-12-16 14:58:51,827 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-16 14:58:51,830 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?), (?, ?) RETURNING id
2022-12-16 14:58:51,835 INFO sqlalchemy.engine.Engine [generated in 0.00024s (insertmanyvalues)] ('squidward', 'Squidward Tentacles', 'ehkrabs', 'Eugene H. Krabs')


Above we observe the Session was first called upon to emit SQL, so it created a new transaction and emitted the appropriate INSERT statements for the two objects. The transaction now remains open until we call any of the Session.commit(), Session.rollback(), or Session.close() methods of Session.

While Session.flush() may be used to manually push out pending changes to the current transaction, it is usually unnecessary as the Session features a behavior known as autoflush, which we will illustrate later. It also flushes out changes whenever Session.commit() is called.

# Autogenerated primary key attributes
Once the rows are inserted, the two Python objects we’ve created are in a state known as persistent, where they are associated with the Session object in which they were added or loaded, and feature lots of other behaviors that will be covered later.

Another effect of the INSERT that occurred was that the ORM has retrieved the new primary key identifiers for each new object; internally it normally uses the same CursorResult.inserted_primary_key accessor we introduced previously. The squidward and krabs objects now have these new primary key identifiers associated with them and we can view them by acesssing the id attribute:

In [24]:
squidward.id
# 4
krabs.id
# 5

2

# Getting Objects by Primary Key from the Identity Map
The primary key identity of the objects are significant to the Session, as the objects are now linked to this identity in memory using a feature known as the identity map. The identity map is an in-memory store that links all objects currently loaded in memory to their primary key identity. We can observe this by retrieving one of the above objects using the Session.get() method, which will return an entry from the identity map if locally present, otherwise emitting a SELECT:

In [26]:
some_squidward = session.get(User, 4)
some_squidward
User(id=4, name='squidward', fullname='Squidward Tentacles')

2022-12-16 15:00:42,144 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-12-16 15:00:42,146 INFO sqlalchemy.engine.Engine [generated in 0.00236s] (4,)


User(id=4, name='squidward', fullname='Squidward Tentacles')

The important thing to note about the identity map is that it maintains a unique instance of a particular Python object per a particular database identity, within the scope of a particular Session object. We may observe that the some_squidward refers to the same object as that of squidward previously:

In [ ]:
some_squidward is squidward
# True

The identity map is a critical feature that allows complex sets of objects to be manipulated within a transaction without things getting out of sync.

# Committing
There’s much more to say about how the Session works which will be discussed further. For now we will commit the transaction so that we can build up knowledge on how to SELECT rows before examining more ORM behaviors and features:

In [27]:
session.commit()
# COMMIT

2022-12-16 15:02:14,130 INFO sqlalchemy.engine.Engine COMMIT


The above operation will commit the transaction that was in progress. The objects which we’ve dealt with are still attached to the Session, which is a state they stay in until the Session is closed (which is introduced at 'Closing a Session').

In [ ]:
sandy = session.execute(select(User).filter_by(name="sandy")).scalar_one()
# BEGIN (implicit)
# SELECT user_account.id, user_account.name, user_account.fullname
# FROM user_account
# WHERE user_account.name = ?
# [...] ('sandy',)
sandy
# User(id=2, name='sandy', fullname='Sandy Cheeks')
sandy.fullname = "Sandy Squirrel"
sandy in session.dirty
# True
sandy_fullname = session.execute(select(User.fullname).where(User.id == 2)).scalar_one()
# UPDATE user_account SET fullname=? WHERE user_account.id = ?
# [...] ('Sandy Squirrel', 2)
# SELECT user_account.fullname
# FROM user_account
# WHERE user_account.id = ?
# [...] (2,)
print(sandy_fullname)
# Sandy Squirrel
sandy in session.dirty
# False

We can see above that we requested that the Session execute a single select() statement. However the SQL emitted shows that an UPDATE were emitted as well, which was the flush process pushing out pending changes. The sandy Python object is now no longer considered dirty:

# Deleting ORM Objects using the Unit of Work pattern

In [ ]:
patrick = session.get(User, 3)
# SELECT user_account.id AS user_account_id, user_account.name AS user_account_name,
# user_account.fullname AS user_account_fullname
# FROM user_account
# WHERE user_account.id = ?
# [...] (3,)
session.delete(patrick)


Current ORM behavior is that patrick stays in the Session until the flush proceeds, which as mentioned before occurs if we emit a query:

In [ ]:
session.execute(select(User).where(User.name == "patrick")).first()
# SELECT address.id AS address_id, address.email_address AS address_email_address,
# address.user_id AS address_user_id
# FROM address
# WHERE ? = address.user_id
# [...] (3,)
# DELETE FROM user_account WHERE user_account.id = ?
# [...] (3,)
# SELECT user_account.id, user_account.name, user_account.fullname
# FROM user_account
# WHERE user_account.name = ?
# [...] ('patrick',)

Above, the SELECT we asked to emit was preceded by a DELETE, which indicated the pending deletion for patrick proceeded. There was also a SELECT against the address table, which was prompted by the ORM looking for rows in this table which may be related to the target row; this behavior is part of a behavior known as cascade, and can be tailored to work more efficiently by allowing the database to handle related rows in address automatically; the section delete has all the detail on this.

Beyond that, the patrick object instance now being deleted is no longer considered to be persistent within the Session, as is shown by the containment check:

In [ ]:
patrick in session
# False

However just like the UPDATEs we made to the sandy object, every change we’ve made here is local to an ongoing transaction, which won’t become permanent if we don’t commit it. As rolling the transaction back is actually more interesting at the moment, we will do that in the next section.